In [1]:
import numpy as np
import scipy.io as sio
import os

In [2]:
#get number of output folders
with open('../system/controlDict') as f:
    startTime = ""
    endTime = ""
    deltaT = ""
    writeInterval = ""
    
    for line in f.readlines():
        if "startTime " in line:
            for m in line:
                if m.isdigit():
                    startTime += m
            startTime = int(startTime)
        
        if "endTime " in line:
            for m in line:
                if m.isdigit():
                    endTime += m
            endTime = int(endTime)
 
        if "deltaT " in line:
            for m in line:
                if m.isdigit() or m == ".":
                    deltaT += m
            deltaT = float(deltaT)

        if "writeInterval " in line:
            for m in line:
                if m.isdigit():
                    writeInterval += m
            writeInterval = int(writeInterval)

folderCount = int((endTime - startTime) / deltaT / writeInterval)

In [3]:
#get number of data points in output files
with open('../1/C') as f:
    lineCount = 1
    lineTotal = ""
    
    for line in f.readlines():
        if "(" in line:
            lineStart = lineCount
            break
        lineTotal = line
        lineCount += 1
    
    lineTotal = int(lineTotal)
    lineEnd = lineStart + int(lineTotal)

In [4]:
#get x and y coordinates
Cx = []
Cy = []

with open('../1/C') as f:
    lineNum = 1
    
    for line in f.readlines():
        if lineNum > lineStart and lineNum <= lineEnd:
            Cx.append(float(line.split()[0][1:]))
            Cy.append(float(line.split()[1]))
        lineNum += 1

Cx_unique = sorted(list(set(Cx)))
Cy_unique = sorted(list(set(Cy)))
Cx_len = len(Cx_unique)
Cy_len = len(Cy_unique)

In [5]:
#get total number of output folders
dataCount = folderCount + 1
count = -1
folderList = []

for folder in os.scandir('../'):
    count += 1
    if folder.is_dir() and count <= dataCount:
        if folder.name.isnumeric():
            folderList.append(int(folder.name))
        else:
            folderList.append(float(folder.name))
        folderList.sort()

timeList = [float(i) for i in folderList]
folderList = folderList[1:]

In [6]:
#prepare array for data
u_data = np.zeros((Cx_len, Cy_len, dataCount))
v_data = np.zeros((Cx_len, Cy_len, dataCount))
p_data = np.zeros((Cx_len, Cy_len, dataCount))

In [7]:
#initial
arr_0 = np.full((750, 50),0.025)

u_data[:,:,0] = arr_0

In [8]:
#pressure
for i in folderList:
    file_name = '../' + str(i) + '/p'
    
    with open(file_name) as f:
        lineNum = 1
        p_temp = []

        for line in f.readlines():
            if lineNum > lineStart and lineNum <= lineEnd:
                p_temp.append(float(line))
            lineNum += 1

    arr_p = np.array(p_temp)
    arr_p = np.reshape(arr_p, (Cy_len, Cx_len)).T
    
    p_data[:,:,i] = arr_p

p_data = np.flip(p_data, 0)

In [9]:
#velocity
for i in folderList:
    file_name = '../' + str(i) + '/U'
    
    with open(file_name) as f:
        lineNum = 1
        u_temp = []
        v_temp = []

        for line in f.readlines():
            if lineNum > lineStart and lineNum <= lineEnd:
                u_temp.append(float(line.split()[0][1:]))
                v_temp.append(float(line.split()[1]))
            lineNum += 1

    arr_u = np.array(u_temp)
    arr_u = np.reshape(arr_u, (Cy_len, Cx_len)).T
    arr_v = np.array(v_temp)
    arr_v = np.reshape(arr_v, (Cy_len, Cx_len)).T
    
    u_data[:,:,i] = arr_u
    v_data[:,:,i] = arr_v
    
u_data = np.flip(u_data, 0)
v_data = np.flip(v_data, 0)

In [10]:
Cy_unique_dimension = [x+(0.025/2) for x in Cy_unique]

t_data = np.array(timeList)
x_data = np.array(Cx_unique)
y_data = np.array(Cy_unique_dimension)

rho = 997
p_data *= rho #multiply by rho to convert to kinematic pressure from static pressure

In [11]:
#nondimensionalize

# rho = 997
# mu = 0.001002
# U_inf = 0.025
# L_norm = 0.025

# u_data /= U_inf
# v_data /= U_inf
# p_data /= (rho*U_inf*U_inf)
# t_data = t_data
# x_data /= L_norm
# y_data /= L_norm

In [12]:
#export data to .mat
sio.savemat('data_CFD_dimension.mat', {'U': u_data, 'V': v_data, 'P': p_data, 't': t_data, 'x': x_data, 'y': y_data})